In [11]:
import os
import tensorflow as tf 
import io
import pandas as pd
from object_detection.utils import dataset_util
from PIL import Image
from time import time

In [12]:
#cwd = os.getcwd()
# new a tensorflow model
sess = tf.Session()
# make a placeholder more flexible
encoded_jpg_ph = tf.placeholder(tf.string, shape=[])
# set resize layer
height = 256
width = 256	
# resizing the image here
decoded_image = tf.image.decode_jpeg(encoded_jpg_ph)
decoded_image_resized = tf.image.resize_images(decoded_image, [height, width]) # this returns float32
decoded_image_resized_uint = tf.cast(decoded_image_resized, tf.uint8)
resize_image   = tf.image.encode_jpeg(decoded_image_resized_uint) # expects uint8
# reset all variables
sess.run(tf.global_variables_initializer())

In [13]:
def create_cat_tf_example(label, label_text, img_path, img_name, row = None):
    """Creates a tf.Example proto from sample cat image.

    Args:
    encoded_cat_image_data: The jpg encoded data of the cat image.

    Returns:
    example: The created tf.Example.
    """

    with tf.gfile.FastGFile(os.path.join(img_path, img_name), 'rb') as fid:
        encoded_image = fid.read() 

    encoded_image_data = sess.run(resize_image, {encoded_jpg_ph: encoded_image}) #  I think this may not be the right way of doing this
    b_filename = str.encode(img_name)

    image_format = b'jpg'
    if not row is None:
        xmins = [row["xmin"]]
        xmaxs = [row["xmax"]]
        ymins = [row["ymin"]]
        ymaxs = [row["ymax"]]
    else:
        xmins = [10.0 / width]
        xmaxs = [(width - 10) / width]
        ymins = [10.0 / height]
        ymaxs = [(height - 10.0) / height]    
    # classes_text = [str.encode(label_text)]
    classes_text = []
    if label_text:
        classes_text.append(label_text.encode('utf8'))
    classes = []
    # if label == 1:
    classes.append(int(label))
    # print(classes_text, classes, b_filename)
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(b_filename),
        'image/source_id': dataset_util.bytes_feature(b_filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        # 'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [14]:
start_time = time()
each_batch_time = time()
box_df = pd.read_csv(os.path.join(".", "auto_box", "auto_box.csv"))


In [15]:
box_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,score
0,dog.0.jpg,499,375,dog,0.157638,0.093376,0.739552,0.805292,97.889382
1,dog.1.jpg,327,499,dog,0.095817,0.096760,0.886062,0.960047,96.295798
2,dog.10.jpg,269,292,dog,0.059631,0.069795,0.923171,0.989220,87.839001
3,dog.100.jpg,500,397,dog,0.071499,0.073221,0.735138,0.987441,95.074093
4,dog.1000.jpg,499,375,dog,0.231863,0.216221,0.583986,0.651290,99.176532


In [16]:
# collect the dirs
mode_list = ["train", "eval"]
for mode in mode_list:
    cwd = os.path.join("C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide", mode)
    writer = tf.python_io.TFRecordWriter("C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/TFRECORD/" + mode + ".tfrecords")
    for index, row in box_df.iterrows():
        class_path = os.path.join(cwd, row["class"])
        img_name = row["filename"]
        name = row["class"]
        if (index % 100 == 0):
            output_str = mode + " step -- " + str(index)
            print(output_str, " compute 100 image_batch time = ", time() - each_batch_time)
            print("id =", int(index+1),name, row["filename"], class_path)
            each_batch_time = time()
        if(os.path.exists(os.path.join(class_path, img_name))):
            if(name == "dog"):
                each_record = create_cat_tf_example(label = 1, label_text = name, img_path = class_path, img_name = img_name, row = row)
            elif (name == "cat"):
                each_record = create_cat_tf_example(label = 2, label_text = name, img_path = class_path, img_name = img_name, row = row)

            writer.write(each_record.SerializeToString())  #序列化为字符串
    writer.close()
    print(mode , "is finished.")
sess.close()
print("cost time =", time() - start_time)

train step -- 0  compute 100 image_batch time =  0.7708053588867188
id = 1 dog dog.0.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 100  compute 100 image_batch time =  0.7734253406524658
id = 101 dog dog.10117.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 200  compute 100 image_batch time =  0.7366945743560791
id = 201 dog dog.10228.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 300  compute 100 image_batch time =  0.7626392841339111
id = 301 dog dog.10346.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 400  compute 100 image_batch time =  0.7384297847747803
id = 401 dog dog.10462.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 500  compute 100 image_batch time =  0.7471904754638672
id = 501 dog dog.10582.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/datase

train step -- 4800  compute 100 image_batch time =  0.7258822917938232
id = 4801 dog dog.4172.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 4900  compute 100 image_batch time =  0.7434487342834473
id = 4901 dog dog.428.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 5000  compute 100 image_batch time =  0.7185714244842529
id = 5001 dog dog.4390.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 5100  compute 100 image_batch time =  0.7341394424438477
id = 5101 dog dog.4504.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 5200  compute 100 image_batch time =  0.7136862277984619
id = 5201 dog dog.4614.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 5300  compute 100 image_batch time =  0.7450485229492188
id = 5301 dog dog.4722.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_det

train step -- 9600  compute 100 image_batch time =  0.7435104846954346
id = 9601 dog dog.955.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 9700  compute 100 image_batch time =  0.7639691829681396
id = 9701 dog dog.9664.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 9800  compute 100 image_batch time =  0.7815783023834229
id = 9801 dog dog.978.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 9900  compute 100 image_batch time =  0.7732934951782227
id = 9901 dog dog.9890.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\dog
train step -- 10000  compute 100 image_batch time =  0.7471253871917725
id = 10001 cat cat.0.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 10100  compute 100 image_batch time =  0.7161037921905518
id = 10101 cat cat.10117.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_de

train step -- 14300  compute 100 image_batch time =  0.7044591903686523
id = 14301 cat cat.3609.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 14400  compute 100 image_batch time =  0.6949965953826904
id = 14401 cat cat.3719.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 14500  compute 100 image_batch time =  0.6894583702087402
id = 14501 cat cat.3830.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 14600  compute 100 image_batch time =  0.7336740493774414
id = 14601 cat cat.3942.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 14700  compute 100 image_batch time =  0.7135448455810547
id = 14701 cat cat.405.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 14800  compute 100 image_batch time =  0.7158920764923096
id = 14801 cat cat.4172.jpg C:/Users/VIPLAB/Desktop/do

train step -- 19000  compute 100 image_batch time =  0.7744014263153076
id = 19001 cat cat.8868.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 19100  compute 100 image_batch time =  0.743654727935791
id = 19101 cat cat.8972.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 19200  compute 100 image_batch time =  0.728327751159668
id = 19201 cat cat.9081.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 19300  compute 100 image_batch time =  0.73455810546875
id = 19301 cat cat.9193.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 19400  compute 100 image_batch time =  0.7001779079437256
id = 19401 cat cat.9311.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\train\cat
train step -- 19500  compute 100 image_batch time =  0.6839721202850342
id = 19501 cat cat.9425.jpg C:/Users/VIPLAB/Desktop/dog_v

...

eval step -- 4100  compute 100 image_batch time =  0.01945042610168457
id = 4101 dog dog.3396.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 4200  compute 100 image_batch time =  0.019030332565307617
id = 4201 dog dog.3499.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 4300  compute 100 image_batch time =  0.01905059814453125
id = 4301 dog dog.3609.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 4400  compute 100 image_batch time =  0.005166053771972656
id = 4401 dog dog.3719.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 4500  compute 100 image_batch time =  0.031368255615234375
id = 4501 dog dog.3830.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 4600  compute 100 image_batch time =  0.018019914627075195
id = 4601 dog dog.3942.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_det

id = 9201 dog dog.9081.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 9300  compute 100 image_batch time =  0.014037609100341797
id = 9301 dog dog.9193.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 9400  compute 100 image_batch time =  0.014038562774658203
id = 9401 dog dog.9311.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 9500  compute 100 image_batch time =  0.013034582138061523
id = 9501 dog dog.9425.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 9600  compute 100 image_batch time =  0.01203155517578125
id = 9601 dog dog.955.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 9700  compute 100 image_batch time =  0.013034582138061523
id = 9701 dog dog.9664.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\dog
eval step -- 9800  compute 100 image

eval step -- 14200  compute 100 image_batch time =  0.03372907638549805
id = 14201 cat cat.3499.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 14300  compute 100 image_batch time =  0.01905059814453125
id = 14301 cat cat.3609.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 14400  compute 100 image_batch time =  0.016754627227783203
id = 14401 cat cat.3719.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 14500  compute 100 image_batch time =  0.015619754791259766
id = 14501 cat cat.3830.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 14600  compute 100 image_batch time =  0.023336410522460938
id = 14601 cat cat.3942.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 14700  compute 100 image_batch time =  0.021082162857055664
id = 14701 cat cat.405.jpg C:/Users/VIPLAB/Desktop/dog

eval step -- 19500  compute 100 image_batch time =  0.0241544246673584
id = 19501 cat cat.9425.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 19600  compute 100 image_batch time =  0.013034582138061523
id = 19601 cat cat.955.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 19700  compute 100 image_batch time =  0.01102900505065918
id = 19701 cat cat.9664.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 19800  compute 100 image_batch time =  0.011029720306396484
id = 19801 cat cat.978.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 19900  compute 100 image_batch time =  0.00913381576538086
id = 19901 cat cat.9890.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 20000  compute 100 image_batch time =  0.0
id = 20001 dog dog.10001.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/d

eval step -- 24300  compute 100 image_batch time =  0.722984790802002
id = 24301 cat cat.6897.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 24400  compute 100 image_batch time =  0.7112360000610352
id = 24401 cat cat.7351.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 24500  compute 100 image_batch time =  0.7174725532531738
id = 24501 cat cat.7764.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 24600  compute 100 image_batch time =  0.7242558002471924
id = 24601 cat cat.8249.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 24700  compute 100 image_batch time =  0.7318069934844971
id = 24701 cat cat.8700.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_detection/dataset/self_divide\eval\cat
eval step -- 24800  compute 100 image_batch time =  0.7355988025665283
id = 24801 cat cat.9186.jpg C:/Users/VIPLAB/Desktop/dog_vs_cat_de